In [17]:
from sentence_transformers import SentenceTransformer
import faiss
from annoy import AnnoyIndex
import numpy as np

In [12]:
schemas = [
    """
    Node properties:
    - **Country**
      - location: POINT
      - code: STRING Example: \"AFG\"
      - name: STRING Example: \"Afghanistan\"
      - tld: STRING Example: \"AF\"
    - **Filing**
      - begin: DATE_TIME Min: 2000-02-08T00:00:00Z, Max: 2017-09-05T00:00:00Z
      - end: DATE_TIME Min: 2000-02-08T00:00:00Z, Max: 2017-11-03T00:00:00Z
      - originator_bank_id: STRING Example: \"cimb-bank-berhad\"
      - sar_id: STRING Example: \"3297\"
      - beneficiary_bank: STRING Example: \"Barclays Bank Plc\"
      - filer_org_name_id: STRING Example: \"the-bank-of-new-york-mellon-corp\"
      - originator_bank_country: STRING Example: \"Singapore\"
      - beneficiary_bank_country: STRING Example: \"United Kingdom\"
      - filer_org_name: STRING Example: \"The Bank of New York Mellon Corp.\"
      - originator_iso: STRING Example: \"SGP\"
      - beneficiary_bank_id: STRING Example: \"barclays-bank-plc-london-england-gbr\"
      - origin_lat: STRING Example: \"1.3667\"
      - origin_lng: STRING Example: \"103.8\"
      - end_date_format: STRING Example: \"2015-09-25T00:00:00Z\"
      - begin_date_format: STRING Example: \"2015-03-25T00:00:00Z\"
      - originator_bank: STRING Example: \"CIMB Bank Berhad\"
      - beneficiary_lat: STRING Example: \"54\"
      - beneficiary_iso: STRING Example: \"GBR\"
      - beneficiary_lng: STRING Example: \"-2\"
      - begin_date: STRING Example: \"Mar 25, 2015\"
      - id: STRING Example: \"223254\"
      - end_date: STRING Example: \"Sep 25, 2015\"
      - amount: INTEGER Min: 1.18, Max: 2721000000
      - number: INTEGER Min: 1, Max: 174
    - **Entity**
      - id: STRING Example: \"the-bank-of-new-york-mellon-corp\"
      - location: POINT
      - name: STRING Example: \"The Bank of New York Mellon Corp.\"
      - country: STRING Example: \"CHN\"
    
    Relationship properties:
    The relationships:
    (:Filing)-[:BENEFITS]->(:Entity)
    (:Filing)-[:CONCERNS]->(:Entity)
    (:Filing)-[:ORIGINATOR]->(:Entity)
    (:Entity)-[:FILED]->(:Filing)
    (:Entity)-[:COUNTRY]->(:Country)
    """,
    """
    Node properties:
- **Person**
- `name`: STRING Example: "Julie Spellman Sweet"
- `id`: STRING Example: "Eaf0bpz6NNoqLVUCqNZPAew"
- `summary`: STRING Example: "CEO at Accenture"
- **Organization**
- `revenue`: FLOAT Example: "1.2E8"
- `motto`: STRING Example: ""
- `nbrEmployees`: INTEGER Example: "375"
- `isDissolved`: BOOLEAN
- `id`: STRING Example: "E0ZU8eCc5OaqS1LU9qE3n3w"
- `isPublic`: BOOLEAN
- `name`: STRING Example: "New Energy Group"
- `summary`: STRING Example: "Software company based in Rome, Metropolitan City "
- **IndustryCategory**
- `name`: STRING Example: "Electronic Products Manufacturers"
- `id`: STRING Example: "EUNd__O4zMNW81lAXNK2GNw"
- **City**
- `id`: STRING Example: "EZHWv2xKgN92oYDKSjhJ2gw"
- `summary`: STRING Example: "City in and county seat of King County, Washington"
- `name`: STRING Example: "Seattle"
- **Country**
- `name`: STRING Example: "United States of America"
- `id`: STRING Example: "E01d4EK33MmCosgI2KXa4-A"
- `summary`: STRING Example: "Country in North America"
- **Article**
- `id`: STRING Example: "ART176872705964"
- `sentiment`: FLOAT Example: "0.856"
- `author`: STRING Example: "David Correa"
- `siteName`: STRING Example: "MrWeb"
- `summary`: STRING Example: "Boston and Mumbai-based consumer behavior analyses"
- `date`: DATE_TIME Example: "2022-03-10T00:00:00Z"
- `title`: STRING Example: "Funds for Consumer Behavior Specialist Infinite An"
- **Chunk**
- `text`: STRING Example: "Boston and Mumbai-based consumer behavior analyses"
- **Fewshot**
- `Question`: STRING Available options: ['How is Emil Eifrem connected to Michael Hunger? ', 'What are the latest news regarding Google? ', 'Are there any news regarding return to office poli', 'What is Microsoft policy regarding to the return t', 'How many levels of subsidiaries does Blackstone ha', 'How many Telecommunications Companies are from Mal']
- `Cypher`: STRING Available options: ['MATCH (p1:Person {{name:"Emil Eifrem"}}), (p2:Pers', 'MATCH (o:Organization {{name:"Google"}})<-[:MENTIO', 'CALL apoc.ml.openai.embedding(["Are there any news', 'CALL apoc.ml.openai.embedding(["What is Microsoft ', 'MATCH p=(o:Organization {name:"Blackstone"})-[:HAS', 'MATCH (o:Organization) WHERE EXISTS {(o)-[:HAS_CAT']
- `id`: INTEGER Min: 2, Max: 7
Relationship properties:

The relationships:
(:Person)-[:HAS_PARENT]->(:Person)
(:Person)-[:HAS_CHILD]->(:Person)
(:Organization)-[:IN_CITY]->(:City)
(:Organization)-[:HAS_CEO]->(:Person)
(:Organization)-[:HAS_CATEGORY]->(:IndustryCategory)
(:Organization)-[:HAS_SUBSIDIARY]->(:Organization)
(:Organization)-[:HAS_SUPPLIER]->(:Organization)
(:Organization)-[:HAS_INVESTOR]->(:Organization)
(:Organization)-[:HAS_INVESTOR]->(:Person)
(:Organization)-[:HAS_BOARD_MEMBER]->(:Person)
(:Organization)-[:HAS_COMPETITOR]->(:Organization)
(:City)-[:IN_COUNTRY]->(:Country)
(:Article)-[:HAS_CHUNK]->(:Chunk)
(:Article)-[:MENTIONS]->(:Organization)
    """
]


In [ ]:
from rank_bm25 import BM25Okapi  
import re


    Node properties:
    - **Country**
      - location: POINT
      - code: STRING Example: "AFG"
      - name: STRING Example: "Afghanistan"
      - tld: STRING Example: "AF"
    - **Filing**
      - begin: DATE_TIME Min: 2000-02-08T00:00:00Z, Max: 2017-09-05T00:00:00Z
      - end: DATE_TIME Min: 2000-02-08T00:00:00Z, Max: 2017-11-03T00:00:00Z
      - originator_bank_id: STRING Example: "cimb-bank-berhad"
      - sar_id: STRING Example: "3297"
      - beneficiary_bank: STRING Example: "Barclays Bank Plc"
      - filer_org_name_id: STRING Example: "the-bank-of-new-york-mellon-corp"
      - originator_bank_country: STRING Example: "Singapore"
      - beneficiary_bank_country: STRING Example: "United Kingdom"
      - filer_org_name: STRING Example: "The Bank of New York Mellon Corp."
      - originator_iso: STRING Example: "SGP"
      - beneficiary_bank_id: STRING Example: "barclays-bank-plc-london-england-gbr"
      - origin_lat: STRING Example: "1.3667"
      - origin_lng: STRING Exa

In [ ]:
class getSchema:
  # Load an embedding model
  def __init__(self):
    self.model = SentenceTransformer("all-MiniLM-L6-v2")
  def get_embedded_schemas(self,schemas):
    self.schema_embeddings = self.model.encode(schemas)

    # Store embeddings in FAISS index
    dimension = self.schema_embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(np.array(self.schema_embeddings))
    return index
  
  def retrieve_schema_faiss(self,schemas,query):
    query_embedding = self.model.encode([query])
    index = self.get_embedded_schemas(schemas)
    distances, indices = index.search(query_embedding, 1)  # Retrieve top match
    best_match = schemas[indices[0][0]]
    return best_match

  def retrieve_schema_annoy(self,schemas,query):
    dimension = 384  # Adjust based on embedding model
    annoy_index = AnnoyIndex(dimension, 'angular')
    self.schema_embeddings = self.model.encode(schemas)
    for i, emb in enumerate(self.schema_embeddings):
        annoy_index.add_item(i, emb)
    annoy_index.build(10)  # Build with 10 trees
    query_embedding = self.model.encode(query)
    index = annoy_index.get_nns_by_vector(query_embedding, 1)[0]
    return schemas[index]

  def retrieve_schema_bm25(self,schemas,query):
    def tokenize(text):
        return re.findall(r'\w+', text.lower())
    tokenized_schemas = [tokenize(schema.lower()) for schema in schemas]  
    bm25 = BM25Okapi(tokenized_schemas)  
    query_tokens = tokenize(query.lower())  
    scores = bm25.get_scores(query_tokens)  
    best_match = schemas[scores.argmax()]  
    return best_match  


  





# Example question
query = "Which schema stores information about financial transactions?"
get_schema_object = getSchema()

print("Best matching schema:\n", get_schema_object.retrieve_schema_annoy(schemas,query))
print("Best matching schema:\n", get_schema_object.retrieve_schema_bm25(schemas,query))
print("Best matching schema:\n", get_schema_object.retrieve_schema_faiss(schemas,query))


Best matching schema:
 
    Node properties:
    - **Country**
      - location: POINT
      - code: STRING Example: "AFG"
      - name: STRING Example: "Afghanistan"
      - tld: STRING Example: "AF"
    - **Filing**
      - begin: DATE_TIME Min: 2000-02-08T00:00:00Z, Max: 2017-09-05T00:00:00Z
      - end: DATE_TIME Min: 2000-02-08T00:00:00Z, Max: 2017-11-03T00:00:00Z
      - originator_bank_id: STRING Example: "cimb-bank-berhad"
      - sar_id: STRING Example: "3297"
      - beneficiary_bank: STRING Example: "Barclays Bank Plc"
      - filer_org_name_id: STRING Example: "the-bank-of-new-york-mellon-corp"
      - originator_bank_country: STRING Example: "Singapore"
      - beneficiary_bank_country: STRING Example: "United Kingdom"
      - filer_org_name: STRING Example: "The Bank of New York Mellon Corp."
      - originator_iso: STRING Example: "SGP"
      - beneficiary_bank_id: STRING Example: "barclays-bank-plc-london-england-gbr"
      - origin_lat: STRING Example: "1.3667"
      -

In [4]:
print(type(index))

<class 'faiss.swigfaiss.IndexFlatL2'>


In [5]:
print(type(index[0]))

TypeError: 'IndexFlatL2' object is not subscriptable